In [1]:
import face_recognition
import cv2
import os
import csv
from datetime import datetime
from time import time

# フォルダ準備
known_dir = "known"
save_dir = "detected_faces"
log_file = "recognition_log.csv"
os.makedirs(known_dir, exist_ok=True)
os.makedirs(save_dir, exist_ok=True)

# CSVログヘッダー（初回のみ書く）
if not os.path.exists(log_file):
    with open(log_file, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["timestamp", "name", "confidence"])

# 顔データ登録
known_face_encodings = []
known_face_names = []

print("🧠 顔画像の登録中...")
for filename in os.listdir(known_dir):
    if filename.endswith((".jpg", ".png")):
        path = os.path.join(known_dir, filename)
        image = face_recognition.load_image_file(path)
        encoding = face_recognition.face_encodings(image)
        if encoding:
            known_face_encodings.append(encoding[0])
            known_face_names.append(os.path.splitext(filename)[0])
            print(f"✅ {filename} 登録成功")
        else:
            print(f"⚠️ {filename} に顔が検出されませんでした")

if not known_face_encodings:
    print("❌ 登録された顔画像がありません。終了します。")
    exit()

# 保存制御
last_saved_time = {}
save_interval_sec = 5

# Webカメラ起動
video_capture = cv2.VideoCapture(0)
print("📸 Webカメラ起動中（qで終了）")

while True:
    ret, frame = video_capture.read()
    if not ret:
        print("❌ 映像取得失敗")
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    current_time = time()

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        name = "Unknown"
        confidence = 0.0

        if known_face_encodings:
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            if len(face_distances) > 0:
                best_match_index = face_distances.argmin()
                confidence = 1 - face_distances[best_match_index]
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.65)
                if matches[best_match_index]:
                    name = known_face_names[best_match_index]

        label_text = f"{name} ({confidence:.2f})"

        # 保存とログ出力制御
        last_time = last_saved_time.get(name, 0)
        if current_time - last_time > save_interval_sec:
            face_image = frame[top:bottom, left:right].copy()
            cv2.putText(face_image, label_text, (5, 25), cv2.FONT_HERSHEY_DUPLEX, 0.8, (0, 0, 0), 2)

            now_str = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"{name}_{now_str}.jpg"
            save_path = os.path.join(save_dir, filename)
            cv2.imwrite(save_path, face_image)

            # 📋 CSVログ追記
            with open(log_file, "a", newline="") as f:
                writer = csv.writer(f)
                writer.writerow([datetime.now().strftime("%Y-%m-%d %H:%M:%S"), name, f"{confidence:.2f}"])

            print(f"💾 保存：{save_path}")
            print(f"📝 ログ記録：{name} @ {now_str}")

            last_saved_time[name] = current_time
        else:
            print(f"⏸️ スキップ：{name}（{int(current_time - last_time)}秒以内）")

        # 表示
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, label_text, (left, top - 10), cv2.FONT_HERSHEY_DUPLEX, 1.0, (0, 0, 0), 2)

    cv2.imshow("Face Recognition with Log", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("🛑 終了します")
        break

video_capture.release()
cv2.destroyAllWindows()

🧠 顔画像の登録中...
✅ ishiba_shigeru.jpg 登録成功
✅ yamada.jpg 登録成功
✅ trump.jpg 登録成功
📸 Webカメラ起動中（qで終了）
💾 保存：detected_faces/yamada_20250506_142625.jpg
📝 ログ記録：yamada @ 20250506_142625
⏸️ スキップ：yamada（0秒以内）
⏸️ スキップ：yamada（1秒以内）
⏸️ スキップ：yamada（1秒以内）
⏸️ スキップ：yamada（1秒以内）
⏸️ スキップ：yamada（2秒以内）
⏸️ スキップ：yamada（2秒以内）
⏸️ スキップ：yamada（2秒以内）
⏸️ スキップ：yamada（3秒以内）
⏸️ スキップ：yamada（3秒以内）
⏸️ スキップ：yamada（3秒以内）
⏸️ スキップ：yamada（4秒以内）
⏸️ スキップ：yamada（4秒以内）
⏸️ スキップ：yamada（4秒以内）
💾 保存：detected_faces/yamada_20250506_142630.jpg
📝 ログ記録：yamada @ 20250506_142630
⏸️ スキップ：yamada（0秒以内）
⏸️ スキップ：yamada（0秒以内）
⏸️ スキップ：yamada（0秒以内）
⏸️ スキップ：yamada（1秒以内）
⏸️ スキップ：yamada（1秒以内）
⏸️ スキップ：yamada（2秒以内）
⏸️ スキップ：yamada（2秒以内）


2025-05-06 14:26:32.663 python3.10[64900:44860876] error messaging the mach port for IMKCFRunLoopWakeUpReliable


⏸️ スキップ：yamada（2秒以内）
⏸️ スキップ：yamada（3秒以内）
⏸️ スキップ：yamada（3秒以内）
⏸️ スキップ：yamada（3秒以内）
⏸️ スキップ：yamada（4秒以内）
⏸️ スキップ：yamada（4秒以内）
⏸️ スキップ：yamada（4秒以内）
💾 保存：detected_faces/yamada_20250506_142635.jpg
📝 ログ記録：yamada @ 20250506_142635
⏸️ スキップ：yamada（0秒以内）
⏸️ スキップ：yamada（0秒以内）
⏸️ スキップ：yamada（1秒以内）
⏸️ スキップ：yamada（1秒以内）
⏸️ スキップ：yamada（1秒以内）
⏸️ スキップ：yamada（2秒以内）
⏸️ スキップ：yamada（2秒以内）
⏸️ スキップ：yamada（2秒以内）
⏸️ スキップ：yamada（3秒以内）
⏸️ スキップ：yamada（3秒以内）
⏸️ スキップ：yamada（3秒以内）
⏸️ スキップ：yamada（4秒以内）
⏸️ スキップ：yamada（4秒以内）
⏸️ スキップ：yamada（4秒以内）
💾 保存：detected_faces/yamada_20250506_142640.jpg
📝 ログ記録：yamada @ 20250506_142640
⏸️ スキップ：yamada（0秒以内）
⏸️ スキップ：yamada（0秒以内）
⏸️ スキップ：yamada（0秒以内）
🛑 終了します
